In [40]:
import pandas as pd


In [41]:
def basic_estimator(pred_class_list, correct_class_list, no_harvest):
    total_classes = 10
    total_cost = 0
    total_data_count = len(pred_class_list)
    valid_data_count = 0
    for i in range(total_data_count):
        pred_class = pred_class_list[i]
        correct_class = correct_class_list[i]
        if pred_class > correct_class or pred_class == 10 or no_harvest:
            if pred_class<correct_class:
                cost = correct_class-pred_class+total_classes
            else:
                cost = pred_class-correct_class
            total_cost += cost
            valid_data_count += 1
            #print "[pred,target,cost]: [{},{},{}]".format(pred_class, correct_class, cost)
    print "total valid data/total data: {}/{}".format(valid_data_count, total_data_count)
    avg_cost = 1.0*total_cost/total_data_count
    print "sum/count = {}/{}={}".format(total_cost, total_data_count, avg_cost)


In [42]:
def ips_plus_estimator(pred_class_list, correct_class_list, logged_class_list, no_harvest, use_logged_action):
    total_classes = 10
    count = 0
    weights = 0
    sum_ = 0
    n = total_data_count = len(pred_class_list)
    count = 0
    count_p1 = 0
    for i in range(n):
        count += 1;
        action_m = pred_class = pred_class_list[i]
        correct_class = correct_class_list[i]
        if pred_class == correct_class and pred_class != total_classes and not no_harvest:
            # correct_class unknown
            continue 
        if pred_class<correct_class:
            cost = correct_class-pred_class+total_classes
        else:
            cost = pred_class-correct_class
        #---- compute p ----
        full_info_action = correct_class
        if use_logged_action:
            a_logging_model = logged_class_list[i]
            if a_logging_model > full_info_action:
                count_p1 += 1
                p = 1
                #print "count (a_logging_model {} > full_info_action {}) is {}".format(a_logging_model,full_info_action,count_p1)
            else:
                p = 0.2
        else:
            if action_m > full_info_action:
                count_p1 += 1
                p = 1
                #print "count (action_m {} > full_info_action {}) is {}".format(action_m,full_info_action,count_p1)
            else:
                p = 0.2
        #---- ---- ---- ----
        sum_ += cost/p
        weights += 1/p
        #print "[pred,target,cost]: [{},{},{}]".format(pred_class, correct_class, cost)
    print "sum/count = {}/{}={}".format(sum_, count, sum_/count)
    print "sum/weights = {}/{}={}".format(sum_, weights, sum_/weights)
    print "p=1 for {}/{}".format(count_p1, n)
        

In [43]:
#==========================================================================#
# Data from deploying M_full                                            #
#==========================================================================#

print "===Deploy M_full==="
dir_name = "results-0522T0733-learning-1-5ms-deploy-full"
log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/0/hvmagent.csv'
df = pd.read_csv(log_path)

correct_class_list = df.cpu_max[1:].reset_index(drop=True)
pred_class_list = df.pred_peak[:-1].reset_index(drop=True) # pred_peak: prediction from learning model
upper_bound_list = df.upper_bound[:-1].reset_index(drop=True) # upper_bound: actual allocation decision "upper_bound = max(pred_peak, current_busy_cores+1)" 

no_harvest = False
print "\nuse upper_bound_list:"
basic_estimator(upper_bound_list, correct_class_list, no_harvest)

===Deploy M_full===

use upper_bound_list:
total valid data/total data: 9423/11856
sum/count = 12654/11856=1.06730769231


In [44]:
#==========================================================================#
# Data from deploying M_partial in shadow mode                             #
#==========================================================================#

print "===Deploy shadow M_partial (ground truth) ==="
dir_name = "results-0524T1645-learning-1-5ms-no-harvest-deploy-partial-max-capped"
log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/0/hvmagent.csv'
df = pd.read_csv(log_path)

correct_class_list = df.cpu_max[1:].reset_index(drop=True)
pred_class_list = df.pred_peak[:-1].reset_index(drop=True) # pred_peak: prediction from learning model
upper_bound_list = df.upper_bound[:-1].reset_index(drop=True) # upper_bound: actual allocation decision "upper_bound = max(pred_peak, current_busy_cores+1)" 

no_harvest = True
print "\nuse upper_bound_list:"
basic_estimator(upper_bound_list, correct_class_list, no_harvest)

===Deploy shadow M_partial (ground truth) ===

use upper_bound_list:
total valid data/total data: 11954/11954
sum/count = 134812/11954=11.2775639953


In [45]:
#==========================================================================#
# Data from deploying M_full with 20% exploration                          #
#==========================================================================#

print "===Deploy M_full (+explore)==="
dir_name = "results-0524T1402-learning-1-5ms-deploy-full-explore"
log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/0/hvmagent.csv'
df = pd.read_csv(log_path)

correct_class_list = df.cpu_max[1:].reset_index(drop=True)
pred_class_list = df.pred_peak[:-1].reset_index(drop=True) # pred_peak: prediction from learning model
upper_bound_list = df.upper_bound[:-1].reset_index(drop=True) # upper_bound: actual allocation decision "upper_bound = max(pred_peak, current_busy_cores+1)" 

no_harvest = False
print "\nuse upper_bound_list:"
avg_cost_eval_actual_run = basic_estimator(upper_bound_list, correct_class_list, no_harvest)

===Deploy M_full (+explore)===

use upper_bound_list:
total valid data/total data: 10385/11788
sum/count = 17388/11788=1.47505938242


In [46]:
#==========================================================================#
# Evaluating M_partial on biased data from deploying M_full (+20% explore) #
#==========================================================================#

# 1. extract biased data from deploying M_full (+explore)
dir_name = "results-0524T1402-learning-1-5ms-deploy-full-explore"
log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/0/hvmagent.csv'
df = pd.read_csv(log_path) 
lines = []
total_data_count = 0
valid_data_count = 0
logged_class_list = []
correct_class_list = []
current_busy_cpu_list = []
for i in range(len(df.iteration)-1):
    correct_class = df.cpu_max[i+1]
    pred_class = df.pred_peak[i]
    upper_bound = df.upper_bound[i]
    current_busy_cpu = df.primary_busy_cores[i]
    total_data_count += 1
    if upper_bound > correct_class or upper_bound==10:
        # overprediction
        logged_class_list.append(max(pred_class, current_busy_cpu+1))
        correct_class_list.append(correct_class)
        current_busy_cpu_list.append(current_busy_cpu)

        valid_data_count += 1
        features = "|busy_cores_prev_interval min:"+str(df.f_min[i])+" max:"+str(df.f_max[i]) \
                    +" avg:"+str(df.f_avg[i])+" stddev:"+str(df.f_stddev[i]) \
                    +" med:"+str(df.f_med[i])
        k = correct_class
        if k<correct_class:
            cost = correct_class-k+10
        else:
            cost = k-correct_class
        label = str(k)+":"+str(cost)+":1 "
        sample = label+features
        lines.append(sample)
print "total data: {}\ntotal valid data: {}".format(total_data_count, valid_data_count)
outF = open("160k/m_partial_biased_data_explore/160k_data.txt", "w")
for line in lines:
  # write line to output file
  outF.write(line)
  outF.write("\n")
outF.close()


# 2. (done in cmd line) use vw test M_partial on biased data and save its predictions 


# 3. read predictions from M_partial on biased data
file = open("160k/m_partial_biased_data_explore/160k_pred_prob.txt", 'r') 
pred_prob_lines = file.readlines() 
pred_class_list = []
upper_bound_list = []
for i in range(len(pred_prob_lines)):
    line = pred_prob_lines[i]
    l = line.strip().split(' ')
    for j in range(10):
        if float(l[j])==1:
            pred=j+1
    upper_bound = pred
    if i==0:
        current_busy_cpu = current_busy_cpu_list[i]
    else:
        current_busy_cpu = min(current_busy_cpu_list[i], upper_bound_list[i-1])
    upper_bound = max(pred, current_busy_cpu+1)
    pred_class_list.append(pred)
    upper_bound_list.append(upper_bound)
    #print "pred_class={}\tupper_bound={}\tcurrent_busy={}\tcorrect_class{}".format(pred, upper_bound,current_busy_cpu,correct_class_list[i])

outF = open("160k/m_partial_biased_data_explore/160k_pred.txt", "w")
for line in pred_class_list:
    outF.write(str(line))
    outF.write("\n")
outF.close()

outF = open("160k/m_partial_biased_data_explore/160k_upper_bound.txt", "w")
for line in upper_bound_list:
    outF.write(str(line))
    outF.write("\n")
outF.close()

    
# 4. run estimators
print "===Evaluating M_partial on biased data (+explore) ==="
no_harvest = True

print "\n*basic estimator*"
print "upper_bound_list"
basic_est = basic_estimator(upper_bound_list, correct_class_list, no_harvest)

print "\n*ips+ estimator*"
print "upper_bound_list"
ips_plus_est = ips_plus_estimator(upper_bound_list, correct_class_list, logged_class_list, no_harvest, False)

total data: 11788
total valid data: 10385
===Evaluating M_partial on biased data (+explore) ===

*basic estimator*
upper_bound_list
total valid data/total data: 10385/10385
sum/count = 103061/10385=9.92402503611

*ips+ estimator*
upper_bound_list
sum/count = 514713.0/10385=49.5631198844
sum/weights = 514713.0/51349.0=10.0238174064
p=1 for 144/10385


In [47]:
#==========================================================================#
# Evaluating M_partial on biased data from deploying M_full                #
#==========================================================================#

# 1. extract biased data from deploying M_full
dir_name = "results-0522T0733-learning-1-5ms-deploy-full"
log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/0/hvmagent.csv'
df = pd.read_csv(log_path) #biased data
lines = []
total_data_count = 0
valid_data_count = 0
logged_class_list = []
correct_class_list = []
current_busy_cpu_list = []
for i in range(len(df.iteration)-1):
    correct_class = df.cpu_max[i+1]
    pred_class = df.pred_peak[i]
    upper_bound = df.upper_bound[i]
    current_busy_cpu = df.primary_busy_cores[i]
    total_data_count += 1
    if upper_bound > correct_class or upper_bound==10:
        # overprediction
        logged_class_list.append(max(pred_class, current_busy_cpu+1))
        correct_class_list.append(correct_class)
        current_busy_cpu_list.append(current_busy_cpu)
        valid_data_count += 1
        features = "|busy_cores_prev_interval min:"+str(df.f_min[i])+" max:"+str(df.f_max[i]) \
                    +" avg:"+str(df.f_avg[i])+" stddev:"+str(df.f_stddev[i]) \
                    +" med:"+str(df.f_med[i])
        k = correct_class
        if k<correct_class:
            cost = correct_class-k+10
        else:
            cost = k-correct_class
        label = str(k)+":"+str(cost)+":1 "
        sample = label+features
        lines.append(sample)
print "total data: {}\ntotal valid data: {}".format(total_data_count, valid_data_count)
outF = open("160k/m_partial_biased_data/160k_data.txt", "w")
for line in lines:
  # write line to output file
  outF.write(line)
  outF.write("\n")
outF.close()


# 2. (done in cmd line) use vw test M_partial on biased data and save its predictions 


# 3. read predictions from M_partial on biased data
file = open('160k/m_partial_biased_data/160k_pred_prob.txt', 'r') 
pred_prob_lines = file.readlines() 
pred_class_list = []
upper_bound_list = []
for i in range(len(pred_prob_lines)):
    line = pred_prob_lines[i]
    l = line.strip().split(' ')
    for j in range(10):
        if float(l[j])==1:
            pred=j+1
    upper_bound = pred
    if i==0:
        current_busy_cpu = current_busy_cpu_list[i]
    else:
        current_busy_cpu = min(current_busy_cpu_list[i], upper_bound_list[i-1])
    upper_bound = max(pred, current_busy_cpu+1)
    pred_class_list.append(pred)
    upper_bound_list.append(upper_bound)
    #print "pred_class={}\tupper_bound={}\tcurrent_busy={}\tcorrect_class{}".format(pred, upper_bound,current_busy_cpu,correct_class_list[i])
    
outF = open("160k/m_partial_biased_data/160k_pred.txt", "w")
for line in pred_class_list:
    outF.write(str(line))
    outF.write("\n")
outF.close()

outF = open("160k/m_partial_biased_data/160k_upper_bound.txt", "w")
for line in upper_bound_list:
    outF.write(str(line))
    outF.write("\n")
outF.close()


# 4. run estimators
print "===Evaluating M_partial on biased data ==="
no_harvest = True

print "\n*basic estimator*"
print "upper_bound_list"
basic_est = basic_estimator(upper_bound_list, correct_class_list, no_harvest)



total data: 11856
total valid data: 9423
===Evaluating M_partial on biased data ===

*basic estimator*
upper_bound_list
total valid data/total data: 9423/9423
sum/count = 99284/9423=10.5363472355
